<a href="https://colab.research.google.com/github/JorgeSedek/tp1_orga_datos/blob/main/C8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


C8) Tenemos la teoría de que a más películas variadas ve un usuario, más snob es, y que en consecuencia da puntajes más bajos en los ratings. Para cada usuario que hizo más de 100 reviews calcule el ratio de compresión de las descripciones de las películas que vió. ¿Cúal es la correlación entre ese ratio y el promedio del rating que deja el usuario?

In [ ]:
df_peliculas = pd.read_csv('/content/drive/MyDrive/orgaDatos/Movie dataset/movies.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_ratings_pelis = pd.read_csv('/content/drive/MyDrive/orgaDatos/Movie dataset/ratings.csv')

In [ ]:
df_ratings_pelis = df_ratings_pelis.rename(columns={"movieId": "id"})

df_peliculas2 = df_peliculas
df_peliculas2['id'] = df_peliculas['id'].map( lambda x: x.replace('-','')).astype('int64')

df = df_ratings_pelis['userId'].value_counts().to_frame().reset_index()
df
df = df.rename(columns={'index':'userId','userId':'cant_user_review'})
df_descripciones = df_peliculas2[['id','overview']]
df_ratings100 = df_ratings_pelis.merge(df, how='outer', on='userId')
df_ratings100 = df_ratings100[df_ratings100['cant_user_review'] > 500]
df_reviews_rating = df_descripciones.merge(df_ratings100, how='outer', on='id')

In [ ]:
df_reviews_rating = df_reviews_rating.dropna()
df_reviews_rating['overview'] = df_reviews_rating['overview'].astype('string')
df_reviews_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3225242 entries, 0 to 3278340
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                int64  
 1   overview          string 
 2   userId            float64
 3   rating            float64
 4   timestamp         float64
 5   cant_user_review  float64
dtypes: float64(4), int64(1), string(1)
memory usage: 172.2 MB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_overview = df_reviews_rating.groupby(['userId'])['overview'].apply(lambda x: ','.join(x)).reset_index()
df_rating_prom = df_reviews_rating.groupby(['userId'])['rating'].mean().reset_index()
df_snob = df_rating_prom.merge(df_overview, how='outer', on='userId')

In [ ]:
from io import BytesIO
import PIL.Image
#import jxlpy
import numpy as np
import gzip
import random
import lzma
import os
from tqdm import tqdm
#import zstandard
import bz2
from functools import lru_cache
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns

def tasa_compresion(x):
  return len(x)/len(gzip.compress(bytes(x, encoding='utf8')))

df_snob['compresion'] = df_snob['overview'].map(lambda x: tasa_compresion(x))

In [ ]:
df_snob[['rating','compresion']].corr()

,rating,compresion
rating,1.000000,-0.164384
compresion,-0.164384,1.000000


la correlación es -0.164384